Filtered Data from Authors: K562_essential_raw_singlecell_01.h5ad
Unfiltered Data from Jasleen: K562_essential_mtx.h5ad



In [1]:
# %reload_ext autoreload
# %autoreload 2
# %aimport czbenchmarks.datasets.utils_single_cell
import czbenchmarks.datasets.utils_single_cell as scp

import os
import scanpy as sc
import pandas as pd
import anndata as ad
import numpy as np
from pandas.testing import assert_frame_equal

## Load Data

In [2]:
filtered_data_path = f"{os.environ['HOME']}/.cz-benchmarks/datasets/replogle_k562_essential_perturbpredict_de_results_control_cells.h5ad"
# filtered_data_path = "/data2/czbenchmarks/replogle2022/K562_essential_raw_singlecell_01.h5ad"
adata_filtered = ad.read_h5ad(filtered_data_path, backed=None)

control_cells_ids = adata_filtered.uns["control_cells_ids"]
control_cells_ids.pop("non-targeting")

array([], dtype=float64)

In [3]:
de_results = {}
de_results['wilcoxon'] = pd.DataFrame(adata_filtered.uns["de_results_wilcoxon"])
de_results['t-test'] = pd.DataFrame(adata_filtered.uns["de_results_t_test"])

## Compare Keys and Shapes

In [4]:
adata_filtered.shape

(310385, 8563)

In [5]:
# The only missing condition is non-targeting (i.e. the control)
# FIXME MICHELLE -- remove control entry from the DE results
for deg_test_name in de_results.keys():
    print(deg_test_name)
    print(
        set(adata_filtered.obs.condition.unique())
        - set(de_results[deg_test_name].condition.unique())
    )
    print(
        set(de_results[deg_test_name].condition.unique())
        - set(adata_filtered.obs.condition.unique())
    )

wilcoxon
{'non-targeting'}
set()
t-test
{'non-targeting'}
set()


## Run DGE Analysis

In [6]:
results = {}
de_keys = ['wilcoxon', 't-test']

In [8]:
for deg_test_name in de_keys:
    print(deg_test_name)
    # num_conditions = 10
    # condition_list = np.asarray(list(control_cells_ids.keys()))
    # condition_list = np.random.choice(condition_list, size=num_conditions, replace=False)
    # new_control_cells_ids = {k: control_cells_ids[k] for k in condition_list}
    new_control_cells_ids = control_cells_ids 


    results[deg_test_name] = scp.run_multicondition_dge_analysis(
        adata=adata_filtered,
        condition_key="condition",
        control_name="non-targeting",
        control_cells_ids=new_control_cells_ids,
        deg_test_name=deg_test_name,
        filter_min_cells=0,  # 0, 10
        filter_min_genes=0,  # 0, 1000
        min_pert_cells=1,  # 1, 50
        remove_avg_zeros=False,
        return_merged_adata=False,
    )

    results[deg_test_name].to_parquet(f"/data2/compute_de_results_{deg_test_name}.arrow")

wilcoxon


Processing de conditions: 100%|██████████████████████████████████████████████████████████████| 2057/2057 [05:34<00:00,  6.14item/s, Completed 2057/2057]


t-test


Processing de conditions:   0%|                                                                                              | 0/2057 [00:00<?, ?item/s]

Processing de conditions:   0%|                                                                    | 1/2057 [00:00<04:08,  8.26item/s, Completed 1/2057]

...

Processing de conditions: 100%|█████████████████████████████████████████████████████████████▉| 2053/2057 [04:19<00:00, 10.24item/s, Completed 2053/2057]

Processing de conditions: 100%|█████████████████████████████████████████████████████████████▉| 2053/2057 [04:20<00:00, 10.24item/s, Completed 2054/2057]

Processing de conditions: 100%|█████████████████████████████████████████████████████████████▉| 2055/2057 [04:20<00:00,  9.30item/s, Completed 2055/2057]

Processing de conditions: 100%|█████████████████████████████████████████████████████████████▉| 2055/2057 [04:20<00:00,  9.30item/s, Completed 2056/2057]

Processing de conditions: 100%|██████████████████████████████████████████████████████████████| 2057/2057 [04:20<00:00,  7.90item/s, Completed 2057/2057]


In [10]:
results['t-test'].head()

,condition,gene_id,score,logfoldchange,pval,pval_adj,standardized_mean_diff
0,ENSG00000001497,ENSG00000265972,4.876804,3.197905,0.000003,0.008658,0.757027
1,ENSG00000001497,ENSG00000143595,4.372360,3.781151,0.000031,0.053487,0.678722
2,ENSG00000001497,ENSG00000175449,4.053357,1.932377,0.000085,0.104590,0.629203
3,ENSG00000001497,ENSG00000127124,3.917709,3.398787,0.000163,0.122874,0.608146
4,ENSG00000001497,ENSG00000270190,3.891927,3.481819,0.000181,0.122874,0.604144


## Or Load from Disk

In [7]:
for deg_test_name in de_keys:
    results[deg_test_name] = pd.read_parquet(f"/data2/compute_de_results_{deg_test_name}.arrow")

In [8]:
for deg_test_name in de_keys:
    print(deg_test_name, results[deg_test_name].shape, adata_cr4.shape)

wilcoxon (19223723, 6) (310385, 32628)


In [12]:
results['wilcoxon'].shape, adata_filtered.shape

((17614091, 6), (310385, 8563))

In [13]:
2057 * 8563

17614091

## Similarity Comparison -- Create Condition - Gene Id Mapper

In [9]:
for deg_test_name in de_keys:
    print(deg_test_name, results[deg_test_name].shape, de_results[deg_test_name].shape)
    print("condition", results[deg_test_name]['condition'].nunique(), de_results[deg_test_name]['condition'].nunique())
    print("gene_id", results[deg_test_name]['gene_id'].nunique(), de_results[deg_test_name]['gene_id'].nunique())

wilcoxon (19223723, 6) (20793007, 7)
condition 1832 2057
gene_id 15730 15730


In [10]:
# Create a column with condition / gene_id to filter the data for similarity
def zip_names(df):
    condition = df['condition'].astype(str).values
    gene_id = df['gene_id'].astype(str).values
    mapper = list(map(lambda x: '_'.join([x[0], x[1]]), zip(condition, gene_id)))
    return mapper

for deg_test_name in de_keys:
    results[deg_test_name]['mapper'] = zip_names(results[deg_test_name])
    de_results[deg_test_name]['mapper'] = zip_names(de_results[deg_test_name])

In [11]:
shared_condition_genes = {}
for deg_test_name in de_keys:
    shared_condition_genes[deg_test_name] = set(results[deg_test_name]['mapper']).intersection(set(de_results[deg_test_name]['mapper']))
    shared_condition_genes[deg_test_name] = list(shared_condition_genes[deg_test_name])

In [12]:
type(shared_condition_genes[deg_test_name]), len(shared_condition_genes[deg_test_name])

(list, 19223723)

In [13]:
s_results = {}
s_de_results = {}

for deg_test_name in de_keys:
    s_results[deg_test_name] = results[deg_test_name].set_index('mapper').loc[shared_condition_genes[deg_test_name]].reset_index(drop=True)
    s_de_results[deg_test_name] = de_results[deg_test_name].set_index('mapper').loc[shared_condition_genes[deg_test_name]].reset_index(drop=True)

## Compare Common Results

In [14]:
common_columns = {}
for deg_test_name in de_keys:
    print(deg_test_name)
    common_columns[deg_test_name] = set(s_de_results[deg_test_name].columns).intersection(s_results[deg_test_name].columns)
    common_columns[deg_test_name] = list(sorted(common_columns[deg_test_name]))
    old_columns = [c for c in s_de_results[deg_test_name].columns if c not in common_columns[deg_test_name]]
    new_columns = [c for c in s_results[deg_test_name].columns if c not in common_columns[deg_test_name]]
    
    with pd.option_context('display.width', 10000, 'display.max_columns', 100):
        print(s_de_results[deg_test_name][common_columns[deg_test_name] + old_columns].head())
        print(s_results[deg_test_name][common_columns[deg_test_name] + new_columns].head())

wilcoxon
         condition          gene_id  logfoldchange      pval  pval_adj     score condition_name
0  ENSG00000115866  ENSG00000113356       0.022121  0.950154  0.999435  0.062514           DARS
1  ENSG00000155666  ENSG00000273802      -0.247788  0.560620  0.999799 -0.581920           KDM8
2  ENSG00000109606  ENSG00000175221      -1.007420  0.000532  0.004872 -3.463998          DHX15
3  ENSG00000095139  ENSG00000136811       0.087385  0.683991  1.000000  0.407024          ARCN1
4  ENSG00000169045  ENSG00000119421       0.177865  0.172091  0.999798  1.365515        HNRNPH1
         condition          gene_id  logfoldchange      pval  pval_adj     score
0  ENSG00000115866  ENSG00000113356       0.022121  0.950154  0.999435  0.062514
1  ENSG00000155666  ENSG00000273802      -0.247788  0.560620  0.999799 -0.581920
2  ENSG00000109606  ENSG00000175221      -1.007420  0.000532  0.004872 -3.463998
3  ENSG00000095139  ENSG00000136811       0.087385  0.683991  1.000000  0.407024
4  ENSG000

In [27]:
# These are the result of running analysis on the raw data
orig_wilcoxon_counts = {}
new_wilcoxon_counts = {}

for deg_test_name in de_keys:
    orig_wilcoxon_counts[deg_test_name] = (
        s_de_results[deg_test_name][["condition", "gene_id"]].groupby("condition").count()
    )
    new_wilcoxon_counts[deg_test_name] = (
        s_results[deg_test_name][["condition", "gene_id"]].groupby("condition").count()
    )
    
    print(deg_test_name)
    print(len(orig_wilcoxon_counts[deg_test_name]), len(new_wilcoxon_counts[deg_test_name]))
    
    unique_conditions = sorted(list(set(orig_wilcoxon_counts[deg_test_name].index).intersection(new_wilcoxon_counts[deg_test_name].index)))
    orig_wilcoxon_counts[deg_test_name] = orig_wilcoxon_counts[deg_test_name].loc[unique_conditions]
    new_wilcoxon_counts[deg_test_name] = new_wilcoxon_counts[deg_test_name].loc[unique_conditions]
    
    print(orig_wilcoxon_counts[deg_test_name].head())
    print(new_wilcoxon_counts[deg_test_name].head())

/tmp/user/1001/ipykernel_436306/4284996731.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  s_de_results[deg_test_name][["condition", "gene_id"]].groupby("condition").count()


wilcoxon
2058 1832
                 gene_id
condition               
ENSG00000001497     9889
ENSG00000004779    10746
ENSG00000004897    12173
ENSG00000005007    10546
ENSG00000005100    10502
                 gene_id
condition               
ENSG00000001497     9889
ENSG00000004779    10746
ENSG00000004897    12173
ENSG00000005007    10546
ENSG00000005100    10502


In [28]:
for deg_test_name in de_keys:
    print(deg_test_name)
    assert_frame_equal(
        orig_wilcoxon_counts[deg_test_name],
        new_wilcoxon_counts[deg_test_name],
        check_dtype=False,
        check_index_type=False,
        check_categorical=False,
    )

wilcoxon


In [32]:
for deg_test_name in de_keys:
    print(deg_test_name)
    print(len(s_de_results[deg_test_name][common_columns[deg_test_name]]), len(s_results[deg_test_name][common_columns[deg_test_name]]))
    assert_frame_equal(
        s_de_results[deg_test_name][common_columns[deg_test_name]],
        s_results[deg_test_name][common_columns[deg_test_name]],
        check_dtype=False,
        check_index_type=False,
        check_categorical=False,
        atol=1e-4
    )

wilcoxon
19223723 19223723


AssertionError: DataFrame.iloc[:, 3] (column name="pval") are different

DataFrame.iloc[:, 3] (column name="pval") values are different (0.03325 %)
[index]: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]
[left]:  [0.950153857543032, 0.5606204865261339, 0.0005322111935425, 0.68399051781166, 0.1720911505157198, 0.9969392219482772, 0.921392773980092, 0.1580455948894672, 0.2602954984994338, 0.7368714326045523, 0.1733138552747071, 0.3151932431314119, 0.5810950290404722, 0.6713197654014221, 0.6597510146580998, 0.7870172193578943, 0.5790260246423479, 0.2404660934721755, 0.5720442708822588, 0.6902298653876919, 0.7927452855893585, 0.5999966170648937, 0

## Misc Checks for Other Files

In [ ]:
# Verify Target Conditions to Save
import json

with open("../target_conditions_to_save.json", "r") as f:
    target_conditions_to_save = json.load(f)

condition_list = set(map(lambda x: x.split("_")[1], target_conditions_to_save.keys()))

target_conditions_to_save_new = {}
for key, value in target_conditions_to_save.items():
    condition = key.split("_")[1]
    if condition not in target_conditions_to_save_new:
        target_conditions_to_save_new[condition] = value
    else:
        assert target_conditions_to_save_new[condition] == value


In [ ]:
# Validate that existing control cells ids 
skipped_conditions = []
# Convert index to set for O(1) lookup instead of O(n) for each check
index_set = set(adata_filtered.obs.index.values)
for key, values in control_cells_ids.items():
    if key not in adata_filtered.obs.condition:
        skipped_conditions.append(key)
    else:
        assert all(x in index_set for x in values), f"{values} not in {adata_filtered.obs.index.values}"

print(f"Skipped conditions: {skipped_conditions}")
